In [ ]:
# dependencies import
from common_dependencies import *
from sklearn.manifold import TSNE
#import model_versions as mv
logger = logging.getLogger(f'main.tsne')

In [ ]:
# train params that can be easily changed
# paths for files with original data
MAX_VAL = 1000
CROP_SIZE = 1
CROP_STEP = 1
XSHIFT = 200

dataset_desc = {'run_1': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT),),
                'run_2': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT),)}

In [ ]:
#входные и выходные данные
# reading
dataset = {'run_1':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_1'])])),
           'run_2':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_2'])]))}

# squueze datasets
'''for dataset_part_name, dataset_part in dataset.items():
    for data_part_name, data_part in dataset_part.items():
        if data_part_name == 'x':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(data_part.shape[0],-1)
        if data_part_name == 'y':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(data_part.shape[0],-1)'''

# displaying
logger.debug('\nDataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}, max={np.max(data_part)}, min={np.min(data_part)}')

In [ ]:
# example of autoencode nondef graphs

FONT_SIZE = 20

for dataset_part_name, dataset_part in dataset.items():
    for i in range(5):
        fig, ax = plt.subplots()
        fig.set_figwidth(12)
        fig.set_figheight(8)
    
        time = dataset_part['x'][dataset_part['bin']!=0][i,0,1:32]
        amp = dataset_part['x'][dataset_part['bin']!=0][i,0,33:]
        
        ax.plot(time, amp, c='black', label='Исходные данные')
        
        ax.set_title(f'Замер датчика в области дефекта',fontsize=FONT_SIZE)
        #ax.legend(fontsize = FONT_SIZE)
        ax.set_xlabel('Время',fontsize=FONT_SIZE)
        ax.set_ylabel('Амплитуда',fontsize=FONT_SIZE)
        ax.tick_params(axis='both', labelsize = FONT_SIZE)
        ax.grid(True)
        #plt.show()
        plt.savefig(f'data/generated_content/all/defect and nondefect plots/{dataset_part_name}/{dataset_part_name}_defect_{i}.jpg', bbox_inches='tight')
        plt.show()
        plt.close()
        
    for i in range(5):
        fig, ax = plt.subplots()
        fig.set_figwidth(12)
        fig.set_figheight(8)
    
        time = dataset_part['x'][dataset_part['bin']==0][i,0,1:32]
        amp = dataset_part['x'][dataset_part['bin']==0][i,0,33:]

        ax.plot(time, amp, c='black', label='Исходные данные')
        
        ax.set_title(f'Замер датчика в области без дефектов',fontsize=FONT_SIZE)
        #ax.legend(fontsize = FONT_SIZE)
        ax.set_xlabel('Время',fontsize=FONT_SIZE)
        ax.set_ylabel('Амплитуда',fontsize=FONT_SIZE)
        ax.tick_params(axis='both', labelsize = FONT_SIZE)
        ax.grid(True)
        #plt.show()
        plt.savefig(f'data/generated_content/all/defect and nondefect plots/{dataset_part_name}/{dataset_part_name}_nondefect_{i}.jpg', bbox_inches='tight')
        plt.show()
        plt.close()

# Новое

In [ ]:
PER = 50
RSTATE = 10
LRATE = 50
NITER = 1500
PATH_SAVE_RESULT = f'data/generated_content/t-SNE models results/crop_size={CROP_SIZE}_crop_step={CROP_STEP}/PER={PER}_RSTATE={RSTATE}_LRATE={LRATE}_NITER={NITER}'

In [ ]:
'''test_x_data = dataset['run_1']['x']
logger.debug(f'{test_x_data.shape=}')

X_embedded = TSNE(n_components=2, 
                  learning_rate=LRATE,
                  init='random', 
                  perplexity=PER,
                  random_state=RSTATE,
                  n_iter=NITER).fit_transform(test_x_data)

logger.debug(f'{X_embedded.shape=}')
df1 = pd.DataFrame(data=X_embedded, columns=['x','y'])
df1['def'] = dataset['run_1']['bin']
df1['x'] = dw.normalize_data(df1['x'].values)
df1['y'] = dw.normalize_data(df1['y'].values)
df1 = df1.sort_values(by=['def'])'''

In [ ]:
test_x_data = np.concatenate([dataset['run_1']['x'], dataset['run_2']['x']], axis=0)
logger.debug(f'{test_x_data.shape=}')

X_embedded = TSNE(n_components=2, 
                  learning_rate=LRATE,
                  init='random', 
                  perplexity=PER,
                  random_state=RSTATE,
                  n_iter=NITER).fit_transform(test_x_data)

X_embedded = dw.normalize_data(X_embedded)
dataset['run_1']['tsne_res'] = X_embedded[:dataset['run_1']['x'].shape[0]]
dataset['run_2']['tsne_res'] = X_embedded[dataset['run_1']['x'].shape[0]:]

logger.debug(f'{X_embedded.shape=}')
logger.debug(f"{dataset['run_1']['tsne_res'].shape=}")
logger.debug(f"{dataset['run_2']['tsne_res'].shape=}")

In [ ]:
df1 = pd.DataFrame(data=dataset['run_1']['tsne_res'], columns=['x','y'])
df1['def'] = dataset['run_1']['bin']
df1['def'] = df1['def'].replace({1:'Дефект',0:'Не дефект'})
#df1['x'] = dw.normalize_data(df1['x'].values)
#df1['y'] = dw.normalize_data(df1['y'].values)
df1 = df1.sort_values(by=['def'], ascending=False)

df2 = pd.DataFrame(data=dataset['run_2']['tsne_res'], columns=['x','y'])
df2['def'] = dataset['run_2']['bin']
df2['def'] = df2['def'].replace({1:'Дефект',0:'Не дефект'})
#df2['x'] = dw.normalize_data(df2['x'].values)
#df2['y'] = dw.normalize_data(df2['y'].values)
df2 = df2.sort_values(by=['def'], ascending=False)

In [ ]:
'''with pd.ExcelWriter(f"{PATH_SAVE_RESULT}/encoding result(only extend df for crop dividing).xlsx") as writer:
    df1.to_excel(writer, sheet_name = 'file_1_encoded')
    df2.to_excel(writer, sheet_name = 'file_2_encoded')'''

In [ ]:
from scipy.stats import gaussian_kde as kde
from matplotlib.colors import Normalize
from matplotlib import cm

def makeColours(samples):
    vals = kde(samples).evaluate(samples)
    norm = Normalize(vmin=vals.min(), vmax=vals.max())
    return [cm.ScalarMappable(norm=norm, cmap='jet').to_rgba(val) for val in vals], vals.min(), vals.max(), vals

'''fig, ax = plt.subplots()
mapp = ax.scatter(samples[0], samples[1], s=1, color=makeColours(dataset['run_1']['tsne_res'].T) )
cbar = fig.colorbar(mapp,ax=ax)
#cbar.ax.tick_params(labelsize=20)
plt.show()'''

In [ ]:
import seaborn as sns

'''g = sns.jointplot(data=df1[df1['def']==0], x='x', y='y', height=10, s=6, 
                  edgecolors='none', marginal_ticks=True)

g.ax_joint.set_xlabel('x', fontsize=15)
g.ax_joint.set_ylabel('y', fontsize=15)
g.ax_joint.legend(fontsize=15)
g.ax_joint.tick_params(axis='both', which='both', labelsize=10)
g.figure.tight_layout() 

g = sns.jointplot(data=df1[df1['def']==1], x='x', y='y', height=10, s=6, 
                  edgecolors='none', marginal_ticks=True)
#g.plot_joint(sns.kdeplot, zorder=1, levels=6)

g.ax_joint.set_xlabel('x', fontsize=15)
g.ax_joint.set_ylabel('y', fontsize=15)
g.ax_joint.legend(fontsize=15)
g.ax_joint.tick_params(axis='both', which='both', labelsize=10)
g.figure.tight_layout() ''';

'''g = sns.jointplot(data=df2, x='x', y='y', height=10, palette=['blue','red'],
                 hue='def', s=6, edgecolors='none', marginal_ticks=True)
#g.plot_joint(sns.kdeplot, zorder=1, levels=6)

g.ax_joint.set_xlabel('x', fontsize=15)
g.ax_joint.set_ylabel('y', fontsize=15)
g.ax_joint.legend(fontsize=15)
g.ax_joint.tick_params(axis='both', which='both', labelsize=10)
g.figure.tight_layout()''';

FONT_SIZE = 15

for name, df in zip(['Файл 1', 'Файл 2', 'Оба файла'],[df1, df2, pd.concat([df1,df2], axis=0)]):
    g = sns.JointGrid(data=df, x="x", y="y", height=10, palette=['gray','black'],
                     hue='def', marginal_ticks=True, xlim=[-0.05,1.05], ylim=[-0.05,1.05], 
                     ratio=2, space=.25)
    g.plot_joint(sns.scatterplot, s=10, edgecolors='none')
    
    g.plot_marginals(sns.histplot, kde=True, bins=40, fill=True)
    #g.plot_marginals(sns.boxplot, medianprops={"color": "white", "linewidth": 1})
    NAME = f'''Отображение признаков из {test_x_data.shape[-1]} на 2-мерное пространство''' #Выделена зона дефектов. 
    g.ax_joint.set_title(NAME, fontsize=FONT_SIZE)
    g.ax_joint.set_xlabel('X', fontsize = FONT_SIZE)
    g.ax_joint.set_ylabel('Y', fontsize = FONT_SIZE)
    g.ax_joint.set_xlim(-0.05,1.05)
    g.ax_joint.set_ylim(-0.05,1.05)
    g.ax_joint.grid(True)
    g.ax_joint.tick_params(axis='both', which='major', labelsize = FONT_SIZE)
    g.ax_joint.tick_params(axis='both', which='minor', labelsize = FONT_SIZE-5)
    g.ax_joint.grid(which='major', color='black', linewidth=0.8)
    g.ax_joint.grid(which='minor', color='gray', linestyle=':', linewidth=0.4)
    g.ax_joint.margins(0.05)
    g.ax_joint.minorticks_on()
    # handles=g.ax_joint.patches, 
    #print(g.ax_joint.artist.get)
    g.ax_joint.legend(title='Класс наблюдения', 
                      #labels=['Дефект', 'Не дефект'],
                      markerscale=2, 
                      alignment='left')
    #g.ax_joint.add_patch(Rectangle((0.1, 0.5), 0.3, 0.25, linewidth=2, edgecolor='white', facecolor='none'))
    g.ax_joint.add_patch(Rectangle((0.1, 0.5), 0.3, 0.25, edgecolor='black', linewidth=2, facecolor='none'))
    
    g.ax_marg_x.set_title('Распределение для точек по оси х', fontsize=FONT_SIZE)
    g.ax_marg_x.legend(title='Класс наблюдения', 
                      labels=['Дефект', 'Не дефект'], 
                      markerscale=4, 
                      alignment='left')
    g.ax_marg_x.grid(True)
    #g.ax_marg_x.set_xlabel('X', fontsize = FONT_SIZE)
    #g.ax_marg_x.set_ylabel('Кол-во точек в диапазоне', fontsize = FONT_SIZE)
    
    
    g.ax_marg_y.set_title('Распределение\nдля точек по оси y', fontsize=FONT_SIZE)
    g.ax_marg_y.legend(title='Класс наблюдения', 
                  labels=['Дефект', 'Не дефект'],
                  markerscale=4, 
                  alignment='left')
    g.ax_marg_y.grid(True)

    
    
    if not os.path.exists(PATH_SAVE_RESULT):
        os.makedirs(PATH_SAVE_RESULT)
    
    #plt.show()
    NAME = NAME.replace('\n','')
    plt.savefig(f'{PATH_SAVE_RESULT}/черно белое/{name}. {NAME} (perpl={PER},rstate={RSTATE},lrate={LRATE},niter={NITER}).jpeg', bbox_inches='tight')
    #g.savefig()
    #g.ax_marg_y.set_xlabel('X', fontsize = FONT_SIZE)
    #g.ax_marg_y.set_ylabel('Кол-во точек в диапазоне', fontsize = FONT_SIZE)
    
#g.ax_marg_x
#g.ax_marg_y

In [ ]:
#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [ ]:
def1_df = df1.query('x >= 0.1 & x <= 0.4 & y >= 0.5 & y <= 0.75')
def2_df = df2.query('x >= 0.1 & x <= 0.4 & y >= 0.5 & y <= 0.75')

In [ ]:
print('file 1')
print('All samples: ', df1.shape)
print('All defect samples: ', df1[df1['def']=='Дефект'].shape)
print('Samples from white range: ', def1_df.shape)
print('Defect samples from white range: ', def1_df[def1_df['def']=='Дефект'].shape)

print('file 2')
print('All samples: ', df2.shape)
print('All defect samples: ', df2[df2['def']=='Дефект'].shape)
print('Samples from white range: ', def2_df.shape)
print('Defect samples from white range: ', def2_df[def2_df['def']=='Дефект'].shape)

In [ ]:
display(def1_df)
display(def1_df[def1_df['def']==1])
display(def2_df)
display(def2_df[def2_df['def']==1])

In [ ]:
def draw_choosen_defects(path_to_data, file_num, path_to_save, choosen_df, xshift, crop_size, crop_step):
    x_df, y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path=path_to_data))

    y_df = dw.roll_df(y_df, xshift, 1)
    y_df = dw.extend_df_for_crops_dividing(y_df, crop_size, crop_step)

    new_y_df = y_df.copy()
    new_y_df.iloc[:,:] = 0
    
    it = iter(sorted(choosen_df.index.to_list()))
    def_ind = next(it)
    cur_ind = 0
    for i in range(0, new_y_df.shape[0] - crop_size + 1, crop_step):  
        for j in range(0, new_y_df.shape[1] - crop_size + 1, crop_step): 
            if cur_ind == def_ind:
                #print(def_ind)
                new_y_df.iloc[i:i+crop_size, j:j+crop_size] += 1
                try:
                    def_ind = next(it)
                except:
                    break
                
            cur_ind += 1

    title = f'Развернутая карта дефектов' # для файла {file_num}
    dw.draw_defects_map(y_df, title=title, path_to_save=f'{path_to_save}/{title} для файла {file_num}.jpeg', pcolormesh_cmap='Greys') #черно белое/
    title = f'Развернутая карта предсказания зон дефектов с помощью t-SNE' # для файла {file_num}
    dw.draw_defects_map(new_y_df, title=title, path_to_save=f'{path_to_save}/{title} для файла {file_num}.jpeg', pcolormesh_cmap='Greys')

In [ ]:
draw_choosen_defects(PATH_TO_DATA['run_1'], 1, PATH_SAVE_RESULT, def1_df, XSHIFT, CROP_SIZE, CROP_STEP)
draw_choosen_defects(PATH_TO_DATA['run_2'], 2, PATH_SAVE_RESULT, def2_df, XSHIFT, CROP_SIZE, CROP_STEP)

In [ ]:
'''import seaborn as sns

FONT_SIZE = 15
PATH_SAVE_RESULT+=f'/PER_{PER}_LRATE_{LRATE}_NITER_{NITER}'
name = f"""Отображения признаков из {test_x_data.shape} на 2 мерное 
(perpl={PER},rand_state={RSTATE},learn_rate={LRATE},n_iter:={NITER},all graphs,all files)"""
SCATTER_SIZE = 2

fig, axes = plt.subplots(2,2)
fig.suptitle(name, fontsize=FONT_SIZE)
fig.set_figwidth(15)
fig.set_figheight(15)

axes[0,0].set_title('Файл 1 (недефектные наблюдения)', fontsize=FONT_SIZE)
axes[0,1].set_title('Файл 1 (дефектные наблюдения)', fontsize=FONT_SIZE)

color, *rang =  makeColours(df1[df1['def']==0][['x','y']].T)
sns.scatterplot(data=df1[df1['def']==0], x='x', y='y', ec="none", s=SCATTER_SIZE, ax=axes[0,0], color=color)
print(rang)

color, *rang =  makeColours(df1[df1['def']==1][['x','y']].T)
sns.scatterplot(data=df1[df1['def']==1], x='x', y='y', ec="none", s=SCATTER_SIZE, ax=axes[0,1], color=color)


axes[1,0].set_title('Файл 2 (недефектные наблюдения)', fontsize=FONT_SIZE)
axes[1,1].set_title('Файл 2 (дефектные наблюдения)', fontsize=FONT_SIZE)
color, *rang =makeColours(df2[df2['def']==0][['x','y']].T)
sns.scatterplot(data=df2[df2['def']==0], x='x', y='y', ec="none", s=SCATTER_SIZE, ax=axes[1,0], color=color)
color, *rang =makeColours(df2[df2['def']==1][['x','y']].T)
sns.scatterplot(data=df2[df2['def']==1], x='x', y='y', ec="none", s=SCATTER_SIZE, ax=axes[1,1], color=color)

for ax in np.array(axes).flatten():
    ax.set_xlabel('X', fontsize = FONT_SIZE)
    ax.set_ylabel('Y', fontsize = FONT_SIZE)
    ax.set_xlim(-0.05,1.05)
    ax.set_ylim(-0.05,1.05)
    ax.grid(True)
    ax.tick_params(axis='both', which='major', labelsize = FONT_SIZE)
    ax.tick_params(axis='both', which='minor', labelsize = FONT_SIZE-5)
    ax.grid(which='major', color='white', linewidth=0.8)
    ax.grid(which='minor', color='white', linewidth=0.4)
    ax.margins(0.05)
    ax.set_facecolor('black')
    ax.minorticks_on()
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.2)

plt.show()

#g = sns.JointGrid() #xlim=[-0.05,1.05], ylim=[-0.05,1.05]
#x, y = dataset['run_2']['tsne_res'][:,0], dataset['run_2']['tsne_res'][:,1]
#sns.scatterplot(x=x, y=y, ec="none", s=2, ax=g.ax_joint, color=makeColours(dataset['run_2']['tsne_res'].T))
#g = sns.JointGrid(data=df1, x="x", y="y", hue="def", palette=['blue','red'])

#g = sns.jointplot(data=df1, x='x', y='y', height=10, s=6, hue='def', edgecolors='none', marginal_ticks=True, palette=['blue','red'], kind='kde', levels=21)
#g = sns.jointplot(data=df1, x='x', y='y', height=10, s=6, hue='def', edgecolors='none', marginal_ticks=True, palette=['blue','red'])
#g = sns.jointplot(data=df2, x='x', y='y', height=10, s=6, hue='def', edgecolors='none', marginal_ticks=True, palette=['blue','red'], kind='kde', levels=21)
#g = sns.jointplot(data=df2, x='x', y='y', height=10, s=6, hue='def', edgecolors='none', marginal_ticks=True, palette=['blue','red'])

#sns.kdeplot(data=df, x="x", hue="def", multiple="fill", ax=g.ax_marg_x)
#sns.kdeplot(data=df, y="y", hue="def", multiple="fill", ax=g.ax_marg_y)
#sns.kdeplot(x=x, linewidth=2, ax=g.ax_marg_x)
#sns.kdeplot(y=y, linewidth=2, ax=g.ax_marg_y)'''

In [ ]:
f'''rom skimage import measure
import seaborn as sns

x_df, y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path=PATH_TO_DATA['run_1']))
# roll data across x axis
x_df = dw.roll_df(x_df, XSHIFT, 1)
y_df = dw.roll_df(y_df, XSHIFT, 1).map(lambda x: 1 if x>0 else 0)

y_arr = y_df.to_numpy()
y_arr_labeled = measure.label(y_arr, connectivity=1)

idx = [np.where(y_arr_labeled == label) for label in np.unique(y_arr_labeled) if label]

print(len(idx))

for arr in idx:
    print(len(arr))
    print(arr[0].shape)

plt.pcolormesh(y_arr)
plt.show()

plt.pcolormesh(y_arr_labeled, cmap = )
plt.show()

dw.draw_defects_map(y_df)
map_labeled = measure.label(y_df.to_numpy(), connectivity=1)
bboxes = [area.bbox for area in measure.regionprops(map_labeled)]
for bbox in bboxes:
    print(bbox)''';

In [ ]:
'''df1 = pd.DataFrame(data=dataset['run_1']['tsne_res'], columns=['x','y'])
df1['def'] = dataset['run_1']['bin']
df1['x'] = dw.normalize_data(df1['x'].values)
df1['y'] = dw.normalize_data(df1['y'].values)

df2 = pd.DataFrame(data=dataset['run_2']['tsne_res'], columns=['x','y'])
df2['def'] = dataset['run_2']['bin']
df2['x'] = dw.normalize_data(df2['x'].values)
df2['y'] = dw.normalize_data(df2['y'].values)'''

In [ ]:
'''FONT_SIZE = 20
PATH_SAVE_RESULT+=f'/PER_{PER}_LRATE_{LRATE}_NITER_{NITER}'
name = f"""Отображения признаков из {test_x_data.shape[1]} мерного пространства (crop_size={CROP_SIZE},crop_step={CROP_STEP}) на 2 мерное 
(perpl={PER},rand_state={RSTATE},learn_rate={LRATE},n_iter:={NITER},all graphs,all files)"""

fig, axes = plt.subplots(1,2)
fig.suptitle(name, fontsize=FONT_SIZE)
fig.set_figwidth(22)
fig.set_figheight(8)

axes[0].set_title('file_1', fontsize=FONT_SIZE)
axes[0].scatter(dataset['run_1']['tsne_res'][dataset['run_1']['bin']==0,0], dataset['run_1']['tsne_res'][dataset['run_1']['bin']==0,1], c='blue', s=3, label='non defect')
axes[0].scatter(dataset['run_1']['tsne_res'][dataset['run_1']['bin']!=0,0], dataset['run_1']['tsne_res'][dataset['run_1']['bin']!=0,1], c='red', s=3, label='defect')

axes[1].set_title('file_2', fontsize=FONT_SIZE)
axes[1].scatter(dataset['run_2']['tsne_res'][dataset['run_2']['bin']==0,0], dataset['run_2']['tsne_res'][dataset['run_2']['bin']==0,1], c='blue', s=3, label='non defect')
axes[1].scatter(dataset['run_2']['tsne_res'][dataset['run_2']['bin']!=0,0], dataset['run_2']['tsne_res'][dataset['run_2']['bin']!=0,1], c='red', s=3, label='defect')

for ax in axes:
    ax.set_xlabel('X',fontsize = FONT_SIZE)
    ax.set_ylabel('Y',fontsize = FONT_SIZE)
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.grid(True)
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    ax.margins(0.05)
    ax.legend(fontsize = FONT_SIZE)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.85, wspace=0.15, hspace=0.2)
name = name.replace("\n","")

if not os.path.exists(PATH_SAVE_RESULT):
    os.makedirs(PATH_SAVE_RESULT)
    
plt.savefig(f'{PATH_SAVE_RESULT}/{name}.jpeg', bbox_inches='tight')
plt.show()'''

In [ ]:
'''mask = [True if (item[0]>=0.45 and item[0]<=0.65 and item[1]>=0.6 and item[1]<=0.8 and def_state!=0) else False 
         for item, def_state in zip(dataset['run_1']['tsne_res'], dataset['run_1']['bin'])]
dataset['run_1']['tsne_def_zone'] = dataset['run_1']['x'][mask]
dataset['run_1']['tsne_def_zone'] = dataset['run_1']['tsne_def_zone'].reshape(-1,CROP_SIZE,CROP_SIZE,64)
logger.debug(f"{dataset['run_1']['tsne_def_zone'].shape=} from total def: {dataset['run_1']['bin'][dataset['run_1']['bin']!=0].shape[0]}")

mask = [True if (item[0]>=0.45 and item[0]<=0.65 and item[1]>=0.6 and item[1]<=0.8 and def_state!=0) else False 
         for item, def_state in zip(dataset['run_2']['tsne_res'], dataset['run_2']['bin'])]
dataset['run_2']['tsne_def_zone'] = dataset['run_2']['x'][mask]
dataset['run_2']['tsne_def_zone'] = dataset['run_2']['tsne_def_zone'].reshape(-1,CROP_SIZE,CROP_SIZE,64)
logger.debug(f"{dataset['run_2']['tsne_def_zone'].shape=} from total def: {dataset['run_2']['bin'][dataset['run_2']['bin']!=0].shape[0]}")'''

In [ ]:
'''def draw_choosen_defects(path_to_data, title, path_to_save, choosen_defects, xshift, crop_size, crop_step):
    x_df, y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path=path_to_data))

    x_df = dw.roll_df(x_df, xshift, 1)
    y_df = dw.roll_df(y_df, xshift, 1)
    
    x_arr = dw.df_to_numpy(x_df)
    y_arr = y_df.to_numpy()
    
    x_arr = np.concatenate([dw.normalize_data(x_arr[:,:,:32]), dw.normalize_data(x_arr[:,:,32:])],axis=2)
    y_arr = dw.normalize_data(y_arr)

    new_y_arr = np.zeros(y_arr.shape)
    print(f'{choosen_defects.shape=}')

    for i in range(0, x_arr.shape[0] - crop_size + 1, crop_step):  
        for j in range(0, x_arr.shape[1] - crop_size + 1, crop_step):  
            if np.sum(y_arr[i:i+crop_size, j:j+crop_size])>0:
                new_y_arr[i:i+crop_size, j:j+crop_size] = 1

    for i in range(0, x_arr.shape[0] - crop_size + 1, crop_step):  
        for j in range(0, x_arr.shape[1] - crop_size + 1, crop_step):  
            for item in choosen_defects:
                if (x_arr[0,0] == item[0,0]).all():
                    new_y_arr[i:i+crop_size, j:j+crop_size] = 2
                    choosen_defects.remove(item)
                    continue
                
    new_y_df =  pd.DataFrame(data=new_y_arr, index=y_df.index, columns=y_df.columns)

    dw.draw_defects_map(y_df, title=title, path_to_save=f'{path_to_save}/{title}')
    dw.draw_defects_map(new_y_df, title=title+' зеленый-все дефекты, желтый-дефект из зоны скопления',
                        path_to_save=f'{path_to_save}/{title} зеленый-все дефекты, желтый-дефект из зоны скопления')
                    '''

In [ ]:
'''draw_choosen_defects(PATH_TO_DATA['run_1'], 'Развернутая карта дефектов для файла 1', PATH_SAVE_RESULT, dataset['run_1']['tsne_def_zone'], XSHIFT, CROP_SIZE, CROP_STEP)
draw_choosen_defects(PATH_TO_DATA['run_2'], 'Развернутая карта дефектов для файла 2', PATH_SAVE_RESULT, dataset['run_2']['tsne_def_zone'], XSHIFT, CROP_SIZE, CROP_STEP)'''

In [ ]:
'''plt.figure()
ax = plt.axes(projection ="3d")
# y_data==0 y_data!=0
name = f'Отображения признаков из {file_1_x.shape[1]} мерного пространства на 2 мерное (perpl: {PER}, rand_state: {RSTATE}, learn_rate: {LRATE}, n_iter: {NITER}, all graphs, file_1)'
plt.scatter(X_embedded[file_1_bin==0,0], X_embedded[file_1_bin==0,1], c='blue', s=1, label='non defect')
plt.scatter(X_embedded[file_1_bin!=0,0], X_embedded[file_1_bin!=0,1], c='red', s=1, label='defect')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.title(name)
plt.grid(True)
#plt.savefig(f'tsne/{name}.jpeg', bbox_inches='tight')
plt.show()''';

In [ ]:
'''fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
#ax.scatter3D(X_embedded[file_1_bin==0,0], X_embedded[file_1_bin==0,1], X_embedded[file_1_bin==0,2], color = "blue", label='non defect')
#ax.scatter3D(X_embedded[file_1_bin!=0,0], X_embedded[file_1_bin!=0,1], X_embedded[file_1_bin!=0,2], color = "red", label='defect')
ax.scatter3D(X_embedded[:,0], X_embedded[:,1], X_embedded[:,2], color = "red", label='defect')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend()
plt.show()''';

In [ ]:
'''plt.figure()
# y_data==0 y_data!=0
name = f'Отображения признаков из {test_x_data.shape[1]} мерного пространства на 2 мерное (perpl: {PER}, rand_state: {RSTATE}, learn_rate: {LRATE}, n_iter: {NITER}, all graphs, file_2)'
plt.scatter(X_embedded[file_1_x.shape[0]:,0], X_embedded[file_1_x.shape[0]:,1], c='red', s=1, label='file_2')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.title(name)
plt.grid(True)
plt.savefig(f'tsne/{name}.jpeg', bbox_inches='tight')''';
#plt.show()

In [ ]:
#def1, nondef1 = file_1_x[file_1_y!=0], file_1_x[file_1_y==0] 
#print(f'{def1.shape=}, {nondef1.shape=}')